In [1]:
HOME_DIR = 'C:/SED/keras-yolo3'
import sys, os
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
#keras는 버전 2.6.0 설치


%matplotlib inline
#notebook을 실행한 브라우저에서 바로 그림을 볼 수 있게 해주는 것


# github을 다운로드한 메인 디렉토리를 sys.path로 잡아준다.
LOCAL_PACKAGE_DIR = os.path.abspath(HOME_DIR) #HOME_DIR 을 abspath 사용해서 절대경로로 잡아줌.
sys.path.append(LOCAL_PACKAGE_DIR)
#sys.path : 모듈을 import 할 때 모듈을 찾아야할 경로들을 저장해둔 list(파이썬 라이브러리들이 저장되어 있는 위치)
#yolo3.model을 사용할 수 있게 경로 지정. (append는 경로 추가.)

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:

LOCAL_PACKAGE_DIR = os.path.abspath(HOME_DIR)
#os.path.abspath : 파이썬 파일의 절대경로.

WEIGHT_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/eighth_test/trained_weights_final.h5')
#학습해서 다운 받은 .h 파일
#.h5 파일은 keras 에서 할 때 모델과 가중치를 모두 넣어준 파일이다.
ANCHOR_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/yolo_anchors.txt')
CLASS_DIR=os.path.abspath(LOCAL_PACKAGE_DIR+'/hardhatvest_classes.txt')

In [3]:
from yolo import YOLO
from PIL import Image
import argparse
sys.path.append(LOCAL_PACKAGE_DIR)
from tensorflow.python.framework.ops import disable_eager_execution
# import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

print(tf.__version__)

2.6.0


In [4]:
helmetvest = YOLO(model_path=WEIGHT_DIR,
            anchors_path=ANCHOR_DIR,
            classes_path=CLASS_DIR)

Instructions for updating:
Colocations handled automatically by placer.
C:\SED\keras-yolo3\eighth_test\trained_weights_final.h5 model, anchors, and classes loaded.


In [5]:
from PyQt5.QtWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *
import sys
import cv2
import numpy as np
import time
import os
import pyautogui
import winsound as sd
from playsound import playsound
from tkinter import messagebox

recent_sec=-1
recent_min=-1

class SED(QWidget): #QWidget : 창 한개 생성
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SED Project") #윈도우 제목
        self.setGeometry(10, 40, 1910, 1080) #기본 창 크기
        self.initUI()

            
    def initUI(self):
        self.cpt = cv2.VideoCapture(0) #웹캠 영상 띄우기
        self.fps = 24
        

        #카메라 연결 시작 (frame은 웹캠에 연결된 상태가 출력되는 영역)
        self.frame = QLabel(self)
        self.frame.resize(1140,830)
        self.frame.setScaledContents(True) #영상이 지정 영역에 맞도록 해서 보여준다.
        self.frame.move(10,10)
        
        #테이블 영역
        self.logtitle=QLabel(self) #라벨로 메시지 생성
        self.logtitle.resize(655,525)
        self.logtitle.move(1310,510)
        self.logtitle.setText("탐지 내역")
        self.logtitle.setFont(QFont("맑은 고딕",12,QFont.Bold))
        
        
        #생성한 버튼을 클릭하면 웹캠 화면이 연결된다.
        self.btn_on = QPushButton("카메라 연결",self)
        self.btn_on.resize(200,25) #버튼 크기
        self.btn_on.move(10,900)
        self.btn_on.setStyleSheet("background-color:#00FFFF;" "font-weight: bold;")
        self.btn_on.clicked.connect(self.start) #클릭할 경우 start 함수와 연결
        
        
        self.btn_off = QPushButton("중지",self) #웹캠 실행 시에 중지 버튼 클릭하면 카메라 종료
        self.btn_off.resize(200,25)
        self.btn_off.move(215,900)
        #move 는 왼쪽 상단 모서리에서 시작해서 (x, y) 좌표
        self.btn_off.setStyleSheet("background-color:#00FFFF;"  "font-weight: bold;" "Color : red")
        self.btn_off.clicked.connect(self.stop) #클릭할 경우 stop 함수와 연결
        
        
        self.btn_off = QPushButton("창 닫기", self) #창 닫기 버튼 클릭하면 창 종료
        self.btn_off.resize(200, 25)
        self.btn_off.move(420,900)
        self.btn_off.setStyleSheet("background-color:#00FFFF;"  "font-weight: bold;")
        self.btn_off.clicked.connect(QCoreApplication.instance().quit)
        
        self.table=QTableWidget(self) #날짜, 시각, 결과를 출력할 테이블 생성
        self.table.resize(680,850)
        self.table.move(1200,40)
        self.table.setRowCount(0) # 처음은 아무것도 없으므로 0
        self.table.setColumnCount(3)
        #테이블에 날짜, 시각, 결과(미착용 헬멧이 몇 개인지) 를 출력하도록 한다
        self.table.setHorizontalHeaderLabels(['날짜','시각','결과'])
        #setHorizontalHeaderLabels(index, 크기 조정) : 컬럼 너비 조정.
        self.table.setColumnWidth(0,self.table.width()*3/10) #날짜
        self.table.setColumnWidth(1,self.table.width()*2/10) #시각
        self.table.setColumnWidth(2,self.table.width()*5/10) #결과

        
        #prt : 상태 메시지 출력
        self.prt=QLabel(self)
        self.prt.resize(200,25)
        self.prt.move(920,900)
        self.prt.setText("대기 중입니다.")#처음에 대기 중입니다를 출력
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : Green")
        #웹캠 실행 시, 객체 탐지된 경우 ""를 출력
        
        #FPS 조절 : 프레임 속도 조절을 위해서 
        self.sldr=QSlider(Qt.Horizontal,self) #슬라이드로 제한된 범위 안에서 값을 조절하는 위젯 생성.
        self.sldr.resize(150,25)
        self.sldr.move(720,900)
        self.sldr.setMinimum(1)
        self.sldr.setMaximum(30)
        self.sldr.setValue(24) #1초에 24장의 이미지로 이루어지도록 설정
        self.sldr.valueChanged.connect(self.setFps)
        
        self.sldrtxt1=QLabel(self)
        self.sldrtxt1.setText("FPS : ")
        self.sldrtxt1.resize(150,25)
        self.sldrtxt1.move(670,900) #FPS 문자
        
        self.fpstxt=QLabel(self)
        self.fpstxt.move(825,880)
        self.fpstxt.setText(str(self.fps)) # 값 준것 텍스트로 (0~30)

        self.show()

    
    def setFps(self):
        self.fps = self.sldr.value() #값 주기
        self.prt.setText("FPS가 "+str(self.fps)+"로 조정.")
        self.prt.setStyleSheet("Color : Black")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.fpstxt.setText(str(self.fps))
        self.timer.stop()
        self.timer.start(1000 / self.fps) #초당 프레임 수 24.

    

    #위에서 한 self.btn_on.clicked.connect(self.start) 에서 start 함수
    def start(self):
        #클릭하면 텍스트 출력하고 웹캠 연결
        self.prt.setText("카메라 연결 중. . .")
        self.prt.setStyleSheet("Color : Red")
        
        
        self.codec = cv2.VideoWriter_fourcc(*'DIVX')
        #fourcc : 4-문자 코드
        
        #VideoWriter : 일련의 프레임을 동영상 파일로 저장
        #코덱 : 인코딩(용량이 큰 파일을 작게 만드는 과정)과 디코딩(본래대로 재생할 수 있게 압축을 푸는 기술)
        #       압축을 하고 또 압축을 푸는 하나의 프로그램
        # DIVX : 시각적 품질을 손상시키지 않고 긴 비디오를 더 작은 크기로 압축하는 기능
        self.timer=QTimer() # 시간 체크해주는 함수.
        self.timer.timeout.connect(self.nextFrameSlot) # 정해준 시간 지나면 nextFrameSlot() 호출
        self.timer.start(int(1000/self.fps))
        #self.codec = cv2.VideoWriter_fourcc(*'XVID')
        #XVID : 비디오 압축을 풀고 압축하기 위해 사용되는 프로그램
        #이것은 저장 공간을 절약하고 파일 전송 속도를 높이기 위해 수행함
        
    def stop(self):
        self.prt.setText("대기 중입니다.")
        self.prt.setStyleSheet("Color : Green")
        self.frame.setPixmap(QPixmap.fromImage(QImage())) 
        #QPixmap.fromImage(QImage())을 이용해서 이미지 라벨에 표시될 이미지로 설정하고 
        #setPixmap 으로 이미지 보여줌.
        self.timer.stop()
            
    
    def nextFrameSlot(self): 
        self.prt.setText("탐지 중. . .")
        self.prt.setFont(QFont("맑은 고딕",10,QFont.Bold))
        self.prt.setStyleSheet("Color : blue")
        _,cam=self.cpt.read() #self.cpt 는 위에서 self.cpt -> cv2.VideoCapture(0)
        cam=cv2.cvtColor(cam,cv2.COLOR_BGR2RGB) #bgr -> rgb로 변환
        #이유 : OpenCV가 이미지를 RGB 형태가 아닌 BGR 형태로 로딩하기 때문에 색감이 다르게 나오기 때문에 변환한다.
        img=Image.fromarray(cam)
        # NumPy 배열로 되어있는 이미지 배열을 PIL 이미지로 변환
        
        detected_img, label = helmetvest.detect_image(img)
        #(helmetvest.detect_image)를 사용하면 이 함수가 알아서 주변의 맴버 함수 이것저것을 사용해서 결국 Detection을 수행한다
        result = np.asarray(detected_img)
        #asarray : 입력을 배열로 변환
        cam = result
        img = QImage(cam, cam.shape[1], cam.shape[0], QImage.Format_RGB888)
        #RGB888: 한줄에 얼만큼의 바이트가 들어가는지를 명시해야 이미지가 깨지는 현상을 방지할 수 있음.
        #8비트가 3채널
        #shape()는 numpy에서 (x, y) 행렬인지 알려줌. / .shape[1]은 열 개수, .shape[0] 은 행 개수.
        
        pix = QPixmap.fromImage(img)
        #QImage를 QPixmap으로 변환 -> imageLabel에 이미지를 보여줄 수 있도록 변환
        
        self.frame.setPixmap(pix)
        #imageLabel의 setPixmap을 통해 이미지를 보여줌
        
        #QImage() : 픽셀단위로 이미지를 다루는 용도로 사용
        #QPixmap : PyQt에서 이미지를 보여줄 때 사용하는 객체, 사진에 있는 포맷들을 지원하는 객체.
        #QPixmap : QPixmap(파일명) 으로 사용함.
        #QImage는 메인메모리상의 이미지이고 QPixmap은 비디오카드상에 존재
        
        t = time.localtime()
        if t.tm_sec % 3 == 0:
            self.prt.setText("탐지 중")
        elif t.tm_sec%3==1:
            self.prt.setText("탐지 중입니다 . . ")
        else:
            self.prt.setText("탐지 중입니다 . . .")
        
        
        if(not(t.tm_sec==recent_sec and t.tm_min == recent_min) and len(label)!=0):
            self.addRow(label)       
        # 초와 분을 처음 부분에서 -1로 초기화 했음.
        # 행을 추가하면서 행을 추가.
        
    def addRow(self,label):
        rowPosition=self.table.rowCount()
        self.table.insertRow(rowPosition)
        t=time.localtime()
        #날짜 (연월일)
        self.table.setItem(rowPosition , 0, QTableWidgetItem("{} / {} / {}".format(t.tm_year, t.tm_mon, t.tm_mday)))
        #시각 (시분초)
        self.table.setItem(rowPosition , 1, QTableWidgetItem("{} : {} : {}".format(t.tm_hour,t.tm_min,t.tm_sec)))
        #데이터 입력하기 위해서 QTableWidgetItem 객체 추가 -> (행, 열, 텍스트) 텍스트 넘김.
        #QTableWidget은 항목들이 표의 형태로 나열되도록 위젯 설정해 주는 기능.
        #QtableWidgetItem() : TableWidget의 항목값을 갖고 있는 객체

        res=""
        if 'helmet' in label:
            res+="착용 : {}건      ".format(len(label['helmet']))
            
        if 'non_helmet' in label:
            res+="헬멧 미착용 : {}건  ".format(len(label['non_helmet']))
            
            sec = 3
            #while은 반복문으로 sec가 0이 되면 반복을 멈춤
            while (sec != 0 ):
                sec = sec-1
                time.sleep(1)
                playsound("c:/SED/hardhat.mp3")
#                안전모를 착용하지 않았습니다. 안전모를 착용 후 다시 와주세요.

            
        if 'vest' in label:
            res+="착용 : {}건      ".format(len(label['vest']))
            
        if 'non_vest' in label:
            res+="조끼 미착용 : {}건  ".format(len(label['non_vest']))
            
            sec = 3
            #while은 반복문으로 sec가 0이 되면 반복을 멈춤
            while (sec != 0 ):
                sec = sec-1
                time.sleep(1)
                playsound("c:/SED/vest.mp3")
#                안전 조끼를 착용하지 않았습니다. 안전조끼를 착용 후 다시 와주세요.

        
        if 'non_helmet' in label and 'non_vest' in label:
            sec = 3
            #while은 반복문으로 sec가 0이 되면 반복을 멈춰라
            while (sec != 0 ):
                sec = sec-1
                time.sleep(1)
                playsound("c:/SED/hardhatvest.mp3")
#                안전모와 안전 조끼를 착용하지 않았습니다. 착용 후 다시 와주세요.
            
            
        self.table.setItem(rowPosition , 2, QTableWidgetItem(res)) #결과 부분
        #setItem(행, 열, 아이템)
        self.table.scrollToBottom() #스크롤 제일 밑으로
        
        global recent_sec
        recent_sec=t.tm_sec
        global recent_min
        recent_min=t.tm_min
        print(recent_sec,recent_min)


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SED Project") #윈도우 제목
        self.setGeometry(10,40,1910,1080) #기본 창 크기
        self.setStyleSheet("background-image : url(c:/SED/title1.jpg)")
        self.w = None
        
        self.button = QPushButton("START", self)
        self.button.setStyleSheet("color: #7FFFD4;")
        self.button.setFont(QFont("맑은 고딕",20,QFont.Bold))
        self.button.resize(400,100)
        self.button.move(750,830)
        self.button.clicked.connect(self.show_new_window)
        

    def show_new_window(self, checked):
        if self.w is None:
            self.w = SED()
            self.w.show()
        else:
            self.w.show()


app = QApplication(sys.argv)
w = MainWindow()
w.show()
app.exec()

C:\Users\park\AppData\Local\Temp\ipykernel_20372\3316061296.py:74: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.table.setColumnWidth(0,self.table.width()*3/10) #날짜
C:\Users\park\AppData\Local\Temp\ipykernel_20372\3316061296.py:75: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.table.setColumnWidth(1,self.table.width()*2/10) #시각
C:\Users\park\AppData\Local\Temp\ipykernel_20372\3316061296.py:76: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.table.setColumnWidth(2,self.table.width()*5/10) #결과


(416, 416, 3)
Found 0 boxes for img
2.631506999999999
(416, 416, 3)
Found 2 boxes for img
non_vest 0.72 (94, 306) (531, 471)
{'non_vest': [[(94, 306), (531, 471)]]}
non_helmet 1.00 (197, 0) (525, 319)
{'non_vest': [[(94, 306), (531, 471)]], 'non_helmet': [[(197, 0), (525, 319)]]}
0.7801548999999994
5 7
(416, 416, 3)
Found 2 boxes for img
non_vest 0.90 (108, 308) (519, 472)
{'non_vest': [[(108, 308), (519, 472)]]}
non_helmet 0.99 (205, 0) (526, 312)
{'non_vest': [[(108, 308), (519, 472)]], 'non_helmet': [[(205, 0), (526, 312)]]}
0.6866593999999964
(416, 416, 3)
Found 2 boxes for img
non_vest 0.95 (119, 306) (515, 474)
{'non_vest': [[(119, 306), (515, 474)]]}
non_helmet 1.00 (209, 0) (530, 317)
{'non_vest': [[(119, 306), (515, 474)]], 'non_helmet': [[(209, 0), (530, 317)]]}
0.6821898000000033
6 7
(416, 416, 3)
Found 2 boxes for img
non_vest 0.74 (99, 308) (530, 471)
{'non_vest': [[(99, 308), (530, 471)]]}
non_helmet 1.00 (207, 0) (527, 311)
{'non_vest': [[(99, 308), (530, 471)]], 'non_he

Found 3 boxes for img
non_vest 0.99 (94, 300) (433, 477)
{'non_vest': [[(94, 300), (433, 477)]]}
non_helmet 0.73 (99, 0) (401, 343)
{'non_vest': [[(94, 300), (433, 477)]], 'non_helmet': [[(99, 0), (401, 343)]]}
helmet 0.99 (78, 7) (383, 306)
{'non_vest': [[(94, 300), (433, 477)]], 'non_helmet': [[(99, 0), (401, 343)]], 'helmet': [[(78, 7), (383, 306)]]}
0.7639063999999962
(416, 416, 3)
Found 3 boxes for img
non_vest 0.99 (96, 302) (435, 479)
{'non_vest': [[(96, 302), (435, 479)]]}
helmet 0.99 (83, 6) (381, 305)
{'non_vest': [[(96, 302), (435, 479)]], 'helmet': [[(83, 6), (381, 305)]]}
0.7641775999999965
32 7
(416, 416, 3)
Found 3 boxes for img
non_vest 0.99 (96, 303) (432, 480)
{'non_vest': [[(96, 303), (432, 480)]]}
non_helmet 0.73 (103, 2) (397, 341)
{'non_vest': [[(96, 303), (432, 480)]], 'non_helmet': [[(103, 2), (397, 341)]]}
helmet 0.99 (84, 7) (382, 305)
{'non_vest': [[(96, 303), (432, 480)]], 'non_helmet': [[(103, 2), (397, 341)]], 'helmet': [[(84, 7), (382, 305)]]}
0.763585800

Found 2 boxes for img
non_vest 0.99 (80, 297) (440, 471)
{'non_vest': [[(80, 297), (440, 471)]]}
helmet 1.00 (153, 0) (408, 278)
{'non_vest': [[(80, 297), (440, 471)]], 'helmet': [[(153, 0), (408, 278)]]}
0.6792383999999885
(416, 416, 3)
Found 2 boxes for img
non_vest 0.98 (75, 298) (444, 470)
{'non_vest': [[(75, 298), (444, 470)]]}
helmet 1.00 (165, 0) (408, 276)
{'non_vest': [[(75, 298), (444, 470)]], 'helmet': [[(165, 0), (408, 276)]]}
0.6826333000000062
56 7
(416, 416, 3)
Found 2 boxes for img
non_vest 0.98 (83, 301) (438, 470)
{'non_vest': [[(83, 301), (438, 470)]]}
helmet 1.00 (170, 0) (407, 275)
{'non_vest': [[(83, 301), (438, 470)]], 'helmet': [[(170, 0), (407, 275)]]}
0.6879553999999928
57 7
(416, 416, 3)
Found 2 boxes for img
non_vest 0.98 (80, 301) (444, 470)
{'non_vest': [[(80, 301), (444, 470)]]}
helmet 1.00 (173, 0) (405, 275)
{'non_vest': [[(80, 301), (444, 470)]], 'helmet': [[(173, 0), (405, 275)]]}
0.6810040000000015
58 7
(416, 416, 3)
Found 2 boxes for img
non_vest 0.

Found 2 boxes for img
non_vest 0.99 (55, 289) (476, 471)
{'non_vest': [[(55, 289), (476, 471)]]}
non_helmet 1.00 (119, 0) (440, 290)
{'non_vest': [[(55, 289), (476, 471)]], 'non_helmet': [[(119, 0), (440, 290)]]}
0.9745848999999964
26 8
(416, 416, 3)
Found 2 boxes for img
non_vest 0.99 (53, 289) (479, 472)
{'non_vest': [[(53, 289), (479, 472)]]}
non_helmet 1.00 (120, 0) (443, 295)
{'non_vest': [[(53, 289), (479, 472)]], 'non_helmet': [[(120, 0), (443, 295)]]}
1.0783149000000094
27 8
(416, 416, 3)
Found 2 boxes for img
non_vest 0.99 (54, 289) (477, 473)
{'non_vest': [[(54, 289), (477, 473)]]}
non_helmet 1.00 (117, 0) (450, 295)
{'non_vest': [[(54, 289), (477, 473)]], 'non_helmet': [[(117, 0), (450, 295)]]}
0.9338157999999908
28 8
(416, 416, 3)
Found 2 boxes for img
non_vest 0.99 (52, 288) (482, 472)
{'non_vest': [[(52, 288), (482, 472)]]}
non_helmet 1.00 (120, 0) (444, 292)
{'non_vest': [[(52, 288), (482, 472)]], 'non_helmet': [[(120, 0), (444, 292)]]}
0.8081985000000032
29 8
(416, 416,

Found 2 boxes for img
non_vest 0.96 (202, 296) (466, 480)
{'non_vest': [[(202, 296), (466, 480)]]}
helmet 0.94 (259, 113) (416, 297)
{'non_vest': [[(202, 296), (466, 480)]], 'helmet': [[(259, 113), (416, 297)]]}
0.7907098000000019
29 9
(416, 416, 3)
Found 2 boxes for img
non_vest 0.97 (194, 296) (457, 480)
{'non_vest': [[(194, 296), (457, 480)]]}
helmet 0.94 (252, 111) (408, 296)
{'non_vest': [[(194, 296), (457, 480)]], 'helmet': [[(252, 111), (408, 296)]]}
0.8054173999999819
30 9
(416, 416, 3)
Found 2 boxes for img
non_vest 0.97 (192, 298) (457, 480)
{'non_vest': [[(192, 298), (457, 480)]]}
helmet 0.93 (252, 112) (406, 292)
{'non_vest': [[(192, 298), (457, 480)]], 'helmet': [[(252, 112), (406, 292)]]}
0.7194980000000157
31 9
(416, 416, 3)
Found 2 boxes for img
non_vest 0.96 (194, 299) (456, 480)
{'non_vest': [[(194, 299), (456, 480)]]}
helmet 0.92 (251, 113) (404, 292)
{'non_vest': [[(194, 299), (456, 480)]], 'helmet': [[(251, 113), (404, 292)]]}
0.7427023999999847
32 9
(416, 416, 3)


Found 1 boxes for img
non_vest 0.94 (326, 178) (582, 474)
{'non_vest': [[(326, 178), (582, 474)]]}
0.863480399999986
1 10
(416, 416, 3)
Found 2 boxes for img
non_vest 0.99 (314, 146) (591, 465)
{'non_vest': [[(314, 146), (591, 465)]]}
non_helmet 0.86 (377, 0) (548, 153)
{'non_vest': [[(314, 146), (591, 465)]], 'non_helmet': [[(377, 0), (548, 153)]]}
0.8759669000000088
2 10
(416, 416, 3)
Found 1 boxes for img
non_vest 0.98 (211, 110) (542, 476)
{'non_vest': [[(211, 110), (542, 476)]]}
0.849033100000014
3 10
(416, 416, 3)
Found 2 boxes for img
non_vest 0.99 (153, 155) (501, 480)
{'non_vest': [[(153, 155), (501, 480)]]}
non_helmet 0.96 (190, 0) (478, 180)
{'non_vest': [[(153, 155), (501, 480)]], 'non_helmet': [[(190, 0), (478, 180)]]}
0.8848562000000015
4 10
(416, 416, 3)
Found 2 boxes for img
non_vest 1.00 (145, 201) (510, 480)
{'non_vest': [[(145, 201), (510, 480)]]}
non_helmet 0.99 (202, 0) (507, 222)
{'non_vest': [[(145, 201), (510, 480)]], 'non_helmet': [[(202, 0), (507, 222)]]}
0.87

0